### Studen't Performance Prediction Model

The model predicts student performance in weeks 3, 5, and 7 of a 16-week CS1 programming course. Starting from the grade, delivery time, and the number of attempts the student generates in programming labs and an exam. 

#### Data Dictionary

| Variable                  | Type             | Description |
|---------------------------|------------------|-------------|
| lab_1                     | decimal          | Grade laboratory 1 |
| delivery_time_lab_1       | decimal          | Delivery time laboratory 1 (days) |
| attempts_lab_1            | integer          | Number of attempts laboratory 1 |
| lab_2                     | decimal          | Grade laboratory 2 |
| lab_3                     | decimal          | Grade laboratory 3 |
| grade                     | integer          | Final grade (0- Low performance; 1- Medium performance; 2- High performance) |

The performance threshold is:

* 0 - Low performance (Final grade between 0.0 and 2.9)
* 1 - Medium performance (Final grade between 3.0 and 4.0)
* 2 - High performance (Final grade between 4.1 and 5.0)

#### Import the libraries

In [1]:
# Import the libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# Algorithms of classification
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
# SVC
from sklearn.svm import SVC
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# K-NN
from sklearn.neighbors import KNeighborsClassifier
# MLP
from sklearn.neural_network import MLPClassifier
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

# Import of library for data split
from sklearn.model_selection import train_test_split
# Confusion matrix
from sklearn.metrics import confusion_matrix
# Classification report
from sklearn.metrics import classification_report
# Import the libraries for the metrics
from sklearn.metrics import precision_score, recall_score, f1_score

# Library to calculate the mean and standard deviation used in the characteristics
from sklearn.preprocessing import StandardScaler
# Library Grid Search
from sklearn.model_selection import GridSearchCV

### Load data to DataFrame

In [2]:
# Load data to DataFrame
data = pd.read_csv("data/classification_data.csv", sep=";")

data

,lab_1,delivery_time_lab_1,attempts_lab_1,lab_2,lab_3,grade
0,4.4,0.63,2,4.4,4.9,2
1,3.3,0.33,1,3.3,3.7,1
2,4.7,0.78,3,4.3,4.8,2
3,4.3,0.28,1,4.0,4.9,2
4,4.2,0.50,10,3.1,4.9,1
...,...,...,...,...,...,...
463,2.2,3.89,6,3.0,3.5,1
464,4.6,0.35,1,0.0,0.0,0
465,4.6,0.48,1,5.0,0.0,1
466,4.6,0.00,0,0.0,0.0,0


### Data preprocessing

In [3]:
# Find NaN records to delete
print('Column         NaN')
print(data.isnull().sum(axis = 0))
print(data.shape)

#data = data.dropna()

Column         NaN
lab_1                  0
delivery_time_lab_1    0
attempts_lab_1         0
lab_2                  0
lab_3                  0
grade                  0
dtype: int64
(468, 6)


In [4]:
# Consult the number of records for qualification
data.groupby('grade').size()

grade
0    162
1    200
2    106
dtype: int64

In [5]:
# Resample
from sklearn.utils import resample

df_low = data[data['grade'] == 0]
df_medium = data[data['grade'] == 1]
df_high = data[data['grade'] == 2]

data_resample_low = resample(df_low,
                replace = True,
                n_samples = 200,
                random_state = 1)

data_resample_high = resample(df_high,
                replace = True,
                n_samples = 200,
                random_state = 1)

data2 = pd.concat([data_resample_low, df_medium, data_resample_high])

data2['grade'].value_counts()


0    200
1    200
2    200
Name: grade, dtype: int64

In [6]:
# DataFrame statistics
data2.describe()

,lab_1,delivery_time_lab_1,attempts_lab_1,lab_2,lab_3,grade
count,600.000000,600.000000,600.000000,600.00000,600.000000,600.000000
mean,2.954000,2.202933,3.060000,3.29350,4.044833,1.000000
std,1.854422,4.200492,4.514092,1.83559,1.590888,0.817178
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1.000000,0.300000,1.000000,1.60000,4.000000,0.000000
50%,3.550000,0.480000,1.000000,4.00000,4.900000,1.000000
75%,4.700000,1.000000,3.000000,4.90000,5.000000,2.000000
max,5.000000,22.080000,39.000000,5.00000,5.000000,2.000000


#### Training and testing set

In [13]:
# Features	
features = ['lab_1','delivery_time_lab_1','attempts_lab_1','lab_2', 'lab_3']
X = data2[features]
# Target variable
y = data2['grade'].values

# The data is split for training (80% training and 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state= 1)

#### Hide warnings

In [14]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

#### Best Features - Eli5

In [16]:
# Model
dtc = DecisionTreeClassifier() 
  
# The model is trained
dtc.fit(X_train, y_train)

pred = dtc.predict(X_test)

# Best Features - Eli5
from eli5 import show_weights

show_weights(dtc, feature_names = features)

Weight,Feature
0.4232,lab_1
0.3835,lab_2
0.1292,lab_3
0.0360,delivery_time_lab_1
0.0281,attempts_lab_1


### --------------------------------------------------------------------------

### Prediction with hyperparameter (Grid Search)

#### Naive Bayes

In [18]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

nb = GaussianNB()

# Parameters
grid = {
    'var_smoothing': np.logspace(0,-9, num=100)
}

grid_search = GridSearchCV(estimator = nb, 
                           param_grid = grid, 
                           cv= 10, 
                           verbose=1,
                           n_jobs=-1,  
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
nb = bestModel

# Train the model with the best hyperparameters
nb.fit(X_train, y_train)

pred = nb.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))



Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Best Parameters (GridSearch): GaussianNB(var_smoothing=0.04328761281083057)
-----------------------------------------------------------
[[29 12  0]
 [10 19  3]
 [ 0  4 43]]
              precision    recall  f1-score   support

           0       0.74      0.71      0.72        41
           1       0.54      0.59      0.57        32
           2       0.93      0.91      0.92        47

    accuracy                           0.76       120
   macro avg       0.74      0.74      0.74       120
weighted avg       0.76      0.76      0.76       120

Precisión:  0.76
Recall:  0.76
F1-Score:  0.76


#### SVC

In [19]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm = SVC()

# Parameters
gamma =  [0.1, 1.0, 10, 100]
C = [0.1, 1.0, 10, 100]
kernel = ['rbf','linear']

grid = dict(gamma = gamma,
            C = C,
            kernel = kernel)

grid_search = GridSearchCV(estimator = svm, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
svm = bestModel
  
# Train the model with the best hyperparameters
svm.fit(X_train, y_train)

pred = svm.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))

Fitting 10 folds for each of 32 candidates, totalling 320 fits
Best Parameters (GridSearch): SVC(C=100, gamma=0.1)
-----------------------------------------------------------
[[39  2  0]
 [ 2 30  0]
 [ 0  3 44]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        41
           1       0.86      0.94      0.90        32
           2       1.00      0.94      0.97        47

    accuracy                           0.94       120
   macro avg       0.94      0.94      0.94       120
weighted avg       0.95      0.94      0.94       120

Precisión:  0.95
Recall:  0.94
F1-Score:  0.94


#### Decision Tree

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

dt = DecisionTreeClassifier()

# Parameters
max_depth = [2, 3, 5, 10, 20]
min_samples_leaf =  [5, 10, 20, 50, 100]
criterion = ["gini", "entropy"]

grid = dict(max_depth = max_depth,
            min_samples_leaf = min_samples_leaf,
            criterion = criterion)

grid_search = GridSearchCV(estimator = dt, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
dtc = bestModel
  
# Train the model with the best hyperparameters
dtc.fit(X_train, y_train)

pred = dtc.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))

Fitting 10 folds for each of 50 candidates, totalling 500 fits
Best Parameters (GridSearch): DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=5)
-----------------------------------------------------------
[[33  8  0]
 [ 3 29  0]
 [ 0  2 45]]
              precision    recall  f1-score   support

           0       0.92      0.80      0.86        41
           1       0.74      0.91      0.82        32
           2       1.00      0.96      0.98        47

    accuracy                           0.89       120
   macro avg       0.89      0.89      0.88       120
weighted avg       0.90      0.89      0.89       120

Precisión:  0.9
Recall:  0.89
F1-Score:  0.89


#### Random Forest

In [28]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

rf = RandomForestClassifier()

# Parameters
bootstrap = [True, False]
max_depth = [10, 20, 50, 100, None]
max_features = ['sqrt', 'log2', None]
min_samples_leaf = [1, 2, 4]
min_samples_split = [2, 5, 10]
n_estimators = [5, 20, 50, 100]

grid = dict(bootstrap = bootstrap, 
            max_depth = max_depth,
            max_features = max_features,
            min_samples_leaf = min_samples_leaf,
            min_samples_split = min_samples_split,
            n_estimators = n_estimators)

grid_search = GridSearchCV(estimator = rf, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
rf = bestModel
  
# Train the model with the best hyperparameters
rf.fit(X_train, y_train)

pred = rf.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))

Fitting 10 folds for each of 1080 candidates, totalling 10800 fits
Best Parameters (GridSearch): RandomForestClassifier(max_depth=10, max_features=None, n_estimators=20)
-----------------------------------------------------------
[[39  2  0]
 [ 3 28  1]
 [ 0  0 47]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94        41
           1       0.93      0.88      0.90        32
           2       0.98      1.00      0.99        47

    accuracy                           0.95       120
   macro avg       0.95      0.94      0.94       120
weighted avg       0.95      0.95      0.95       120

Precision:  0.95
Recall:  0.95
F1-Score:  0.95


#### Logistic Regression

In [24]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

lr = LogisticRegression()

# Parameters
solver = ['lbfgs','newton-cg','liblinear']
penalty = ['l2']
C = [100, 10, 1.0, 0.1, 0.01]
max_iter = [100, 1000,2500, 5000]

grid = dict(solver = solver,
            penalty = penalty,
            C = C,
            max_iter = max_iter)

grid_search = GridSearchCV(estimator = lr, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
lr = bestModel
  
# Train the model with the best hyperparameters
lr.fit(X_train, y_train)

pred = lr.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))

Fitting 10 folds for each of 60 candidates, totalling 600 fits
Best Parameters (GridSearch): LogisticRegression(C=100)
-----------------------------------------------------------
[[31 10  0]
 [ 1 30  1]
 [ 0  0 47]]
              precision    recall  f1-score   support

           0       0.97      0.76      0.85        41
           1       0.75      0.94      0.83        32
           2       0.98      1.00      0.99        47

    accuracy                           0.90       120
   macro avg       0.90      0.90      0.89       120
weighted avg       0.91      0.90      0.90       120

Precision:  0.91
Recall:  0.9
F1-Score:  0.9


#### K-NN

In [25]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier()

# Parameters
n_neighbors = [1, 3, 5, 10]
weights = ['uniform','distance']
algorithm = ['auto','ball_tree','kd_tree','brute']

grid = dict(n_neighbors = n_neighbors,
            weights = weights,
            algorithm = algorithm)

grid_search = GridSearchCV(estimator = knn, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
lr = bestModel
  
# Train the model with the best hyperparameters
lr.fit(X_train, y_train)

pred = lr.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))

Fitting 10 folds for each of 32 candidates, totalling 320 fits


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be e

Best Parameters (GridSearch): KNeighborsClassifier(n_neighbors=1)
-----------------------------------------------------------
[[37  4  0]
 [ 2 29  1]
 [ 0  1 46]]
              precision    recall  f1-score   support

           0       0.95      0.90      0.92        41
           1       0.85      0.91      0.88        32
           2       0.98      0.98      0.98        47

    accuracy                           0.93       120
   macro avg       0.93      0.93      0.93       120
weighted avg       0.93      0.93      0.93       120

Precision:  0.93
Recall:  0.93
F1-Score:  0.93


#### MLP

In [26]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

mlp = MLPClassifier(max_iter=150)

# Parameters
hidden_layer_sizes = [(10,),(20,),(50,),(100,)]
activation = ['tanh', 'relu']
solver = ['sgd', 'adam']
alpha = [0.0001, 0.05]
learning_rate =  ['constant','adaptive']

grid = dict(hidden_layer_sizes = hidden_layer_sizes,
            activation = activation, 
            solver = solver,
            alpha = alpha,
            learning_rate = learning_rate)

grid_search = GridSearchCV(estimator = mlp, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
lr = bestModel
  
# Train the model with the best hyperparameters
lr.fit(X_train, y_train)

pred = lr.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))

Fitting 10 folds for each of 64 candidates, totalling 640 fits


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) 

Best Parameters (GridSearch): MLPClassifier(activation='tanh', hidden_layer_sizes=(50,), max_iter=150)
-----------------------------------------------------------
[[31 10  0]
 [ 2 29  1]
 [ 0  0 47]]
              precision    recall  f1-score   support

           0       0.94      0.76      0.84        41
           1       0.74      0.91      0.82        32
           2       0.98      1.00      0.99        47

    accuracy                           0.89       120
   macro avg       0.89      0.89      0.88       120
weighted avg       0.90      0.89      0.89       120

Precision:  0.9
Recall:  0.89
F1-Score:  0.89


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (150) reached and the optimization hasn't converged yet.
  warnings.warn(


#### GBC

In [27]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

gbc = GradientBoostingClassifier()

# Parameters
learning_rate = [0.01, 0.05, 0.1, 0.15, 0.2]
criterion = ['friedman_mse', 'squared_error']
max_depth = [3,5,8]
max_features = ['log2','sqrt']

grid = dict(learning_rate = learning_rate,
            criterion = criterion,
            max_depth = max_depth,
            max_features = max_features)

grid_search = GridSearchCV(estimator = gbc, 
                           param_grid = grid, 
                           cv= 10,  
                           verbose=1, 
                           n_jobs=-1,
                           scoring = "accuracy")

searchResults = grid_search.fit(X_train, y_train.ravel())

# Extract the best model and evaluate it
bestModel = searchResults.best_estimator_

print("Best Parameters (GridSearch):", bestModel)
print("-----------------------------------------------------------")

# Object best hyperparameters
gbc = bestModel
  
# Train the model with the best hyperparameters
gbc.fit(X_train, y_train)

pred = gbc.predict(X_test)

# Confusion matrix
print(confusion_matrix(y_test, pred))
# Classification report
print(classification_report(y_test, pred))

# Metrics: Precision, Recall, F1-Score
print("Precision: ", round(precision_score(y_test, pred, average='weighted'), 2))
print("Recall: ", round(recall_score(y_test, pred, average='weighted'),2))
print("F1-Score: ", round(f1_score(y_test, pred, average='weighted'),2))

Fitting 10 folds for each of 60 candidates, totalling 600 fits
Best Parameters (GridSearch): GradientBoostingClassifier(learning_rate=0.2, max_features='sqrt')
-----------------------------------------------------------
[[38  3  0]
 [ 2 30  0]
 [ 0  0 47]]
              precision    recall  f1-score   support

           0       0.95      0.93      0.94        41
           1       0.91      0.94      0.92        32
           2       1.00      1.00      1.00        47

    accuracy                           0.96       120
   macro avg       0.95      0.95      0.95       120
weighted avg       0.96      0.96      0.96       120

Precision:  0.96
Recall:  0.96
F1-Score:  0.96
